In [ ]:
import sys
sys.path.append('../')

import torch
import pandas as pd
from scipy.sparse.linalg import spsolve

from fit_heat_multibc import C2DS, C5DS
from UniformICD import UniformFVM
from Problems.BlockSourceProblem import BlockSourceProblem
from utils import layout2csv, ChipLayout
from MyPlot import *
from BaseTester import BaseTester


In [ ]:
class HeatTester(BaseTester):
	def __init__(self,  **kwargs):
		super().__init__(**kwargs)
		self.boundary_gap = 0.001
		self.chip_gap = 0.001
		self.mesh()
		self.gen_linsys()

		self.h = 0.1 / self.GridSize
	
	def l2(self, pre, ans):
		return np.sqrt(((pre - ans)**2 * self.h**2).sum())
			
	def gen_layouts(self, DataN):
		df = layout2csv(DataN, self.area, self.GridSize, self.boundary_gap, self.chip_gap)
		layouts = []

		for _, data in df.groupby("idx"):
			info = data.values[:, 1:]
			chips_layout = ChipLayout(info)
			layout = chips_layout(self.xx, self.yy)
			layouts.append(torch.from_numpy(layout))
		layouts = torch.stack(layouts, dim=0)
		return layouts

	def gen_linsys(self):
		self.A, self.B = [], []
		for bd_case in [0, 1, 2]:
			problem = BlockSourceProblem(
					None, 
					bd_case, 298, 
					self.GridSize, 
					self.area
			)

			solver = UniformFVM(
				self.area, 
				self.GridSize, 
				self.GridSize, 
				problem
			)
			
			A, b = solver.get_A()
			
			self.B.append(b)
			self.A.append(A)

	def init_test_ds(self, bd_cases):
		DataN = len(bd_cases)
		layouts = self.gen_layouts(DataN)
		match self.kwargs['net_kwargs']['in_channels']:
			case 2:
				ds = C2DS(
					self.area, 
					self.GridSize, 
					layouts, self.dtype, self.device, bd_cases
				)
			case 5:
				ds = C5DS(
					self.area, 
					self.GridSize, 
					layouts, self.dtype, self.device, bd_cases
				)
		self.ds = ds
    
	def test(self, exp_name, bd_cases, best_or_last):
		self.load_kwargs(exp_name)
		self.init_test_ds(bd_cases)
		self.load_ckpt(best_or_last, exp_name)

		df = {
			'id': [],
			'l2': []
		}

		with torch.no_grad():
			for i, (data, layout, boundary, bd_case) in enumerate(self.ds):
				data = data[None, ...]

				if 'Plus298' in exp_name:
					pre = self.net(data) + 298
				else:
					pre = self.net(data) 
								
				b = layout.cpu().numpy().reshape(-1) * self.dx * self.dy + self.B[bd_case]
				A = self.A[bd_case]

				ans = spsolve(A, b).reshape(self.GridSize, self.GridSize)
				l2 = self.l2(pre.cpu().numpy(), ans)
				if l2 >= 0.01:
					continue
				df['id'].append(i)
				df['l2'].append(l2)
				self.save_img(f"{self.img_save_path}/{exp_name}/TestCase-{i}", pre, ans, boundary, layout)
		
		df = pd.DataFrame(df)
		df.to_csv(f"{self.img_save_path}/{exp_name}/l2.csv", index=False)
		
	def save_img(self, path, pre, ans, boundary, Force):
		p = Path(path)
		if not p.is_dir():
				p.mkdir(parents=True)

		pre = pre.cpu().numpy().reshape(self.GridSize, self.GridSize)
		ans = ans.reshape(self.GridSize, self.GridSize)
		boundary = boundary.cpu().numpy().reshape(self.GridSize, self.GridSize)
		Force = Force.cpu().numpy().reshape(self.GridSize, self.GridSize)

		save_surf(path, pre, self.xx, self.yy, 'surf_pre')
		save_surf(path, ans, self.xx, self.yy, 'surf_ref')
		save_ctf(path, pre, ans, self.xx, self.yy, vmin=0.0, vmax=0.5)
		save_contour(path, pre, ans, self.xx, self.yy, levels=None)
		save_img_force(path, Force, 'force')
		save_img_force(path, boundary, 'boundary')

In [ ]:
heat_tester = HeatTester(
    GridSize=128,
    area=((0, 0), (0.1, 0.1)),
    ckpt_save_path=f'model_save',
    hyper_parameters_save_path = f'hyper_parameters', 
    img_save_path = f'./images', 
    device='cuda',
    dtype=torch.float,
    )

exp_name='JJQC2-ResBottleNeck-2#4#4#6#8-2#4#4#6#8-2-layer-max-replicate-128-jac-15-10000-5'
# exp_name = 'JJQC2Plus298-ResBottleNeck-2#4#4#6#8-2#4#4#6#8-2-layer-max-replicate-128-jac-15-10000-5'
best_or_last='last'

# bd_cases = np.random.choice([0, 1, 2], 3)
bd_cases = [0, 1, 2]
# bd_cases = [2] * 10
heat_tester.test(exp_name, bd_cases, best_or_last)